# **RISK ASSESSMENT USING RFA AND TRADING VIEW PINESCRIPT INDICATORS**

In [1]:
!pip install yfinance
!pip install pandas_ta
!pip install joblib
!pip install traceback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=95b350598f2cc98e520bbd07d10b9ba44d86368b1df10071d16c9ba4c4706c3e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
ERROR: Could not find a version that satisfies the requirement traceback (from versions: none)
ERROR: No matching distribution found for traceback


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import joblib
import os
import logging
import traceback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

def get_stock_data(ticker, period='1y', interval='1d'):
    """
    Fetch historical stock data for a given ticker using Yahoo Finance API.
    Dynamically adjusts the period for newly listed stocks if data is unavailable.
    """
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period, interval=interval)

        # If data is not available for the specified period, try shorter periods
        if hist.empty:
            print(f"Data for {ticker} not available for period '{period}'. Trying shorter periods.")
            for fallback_period in ['6mo', '3mo', '1mo']:
                hist = stock.history(period=fallback_period, interval=interval)
                if not hist.empty:
                    print(f"Data found for {ticker} with period '{fallback_period}'.")
                    break

        if hist.empty:
            raise ValueError(f"No data found for ticker: {ticker}")

        print(hist.head())
        return hist

    except Exception as e:
        logging.error(f"Error fetching data for ticker {ticker}: {e}")
        raise ValueError(f"Failed to fetch data for {ticker}: {e}")


    except Exception as e:
        logging.error(f"Error fetching data for ticker {ticker}: {e}")
        raise ValueError(f"Failed to fetch data for {ticker}: {e}")


def preprocess_data(df):
    """
    Preprocess the stock data by handling missing values and resetting the index.
    """
    try:
        if df.empty or len(df) < 10:  # Ensure enough data points exist
            raise ValueError("Insufficient data available for meaningful analysis.")

        df.dropna(inplace=True)  # Remove missing values
        df.reset_index(inplace=True)  # Reset index
        print("\nPREPROCESSED DATA")
        print(df.head())
        print()
        return df
    except Exception as e:
        raise ValueError(f"Error during preprocessing: {e}")


def add_features(df):
    """
    Add technical indicators as features for the model. Handles missing data appropriately.
    """
    try:
        # Basic features
        df['Daily Return'] = df['Close'].pct_change()
        df['Volatility'] = df['Daily Return'].rolling(window=21).std() * np.sqrt(252)
        df['MA50'] = df['Close'].rolling(window=50).mean()
        df['MA200'] = df['Close'].rolling(window=200).mean()

        # Additional technical indicators using pandas_ta
        df['RSI'] = df.ta.rsi(length=14)
        macd = df.ta.macd(fast=12, slow=26, signal=9)
        df['MACD'] = macd['MACD_12_26_9']

        # Bollinger Bands
        bb_bands = df.ta.bbands(close=df['Close'], length=20)
        df['BB_upper'] = bb_bands['BBU_20_2.0']
        df['BB_middle'] = bb_bands['BBM_20_2.0']
        df['BB_lower'] = bb_bands['BBL_20_2.0']

        # Handle missing values after feature addition
        df.dropna(inplace=True)

        print("\nADDED TECHNICAL INDICATORS AS FEATURES")
        print(df.head())
        print()

        return df
    except Exception as e:
        raise ValueError(f"Error adding features: {e}")


def label_risk(df):
    """
    Label the risk level based on volatility quantiles.
    """
    try:
        df = df.dropna(subset=['Volatility'])  # Ensure no missing values in Volatility
        quantiles = df['Volatility'].quantile([0.33, 0.66])
        conditions = [
            (df['Volatility'] > quantiles[0.66]),
            (df['Volatility'] <= quantiles[0.66]) & (df['Volatility'] > quantiles[0.33]),
            (df['Volatility'] <= quantiles[0.33])
        ]
        choices = ['High', 'Medium', 'Low']
        df['Risk Level'] = np.select(conditions, choices)
        return df
    except Exception as e:
        raise ValueError(f"Error during labeling: {e}")

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score

def train_and_save_model(ticker, model_path, scaler_path):
    """
    Train a machine learning model for a specific ticker, optimize hyperparameters using GridSearchCV,
    and save the model along with its scaler.
    """
    try:
        # Fetch, preprocess, and add features to the stock data
        data = get_stock_data(ticker)
        data = preprocess_data(data)
        data = add_features(data)
        data = label_risk(data)

        # Select features and target
        features = ['Daily Return', 'Volatility', 'MA50', 'MA200']
        X = data[features].values
        y = data['Risk Level'].values.ravel()

        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Define parameters for grid search
        parameters = [{'n_estimators': [100, 700],
                       'max_features': ['sqrt', 'log2'],
                       'criterion': ['gini', 'entropy']}]

        # Perform Grid Search with cross-validation
        grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                                   parameters,
                                   cv=5,
                                   scoring='accuracy',
                                   n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)

        # Retrieve the best model and parameters
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        print(f"Best Accuracy (Grid Search): {grid_search.best_score_ * 100:.2f}%")
        print(f"Best Parameters: {best_params}")

        # Fit the model with the best parameters
        best_model.fit(X_train_scaled, y_train)

        # Evaluate the model on the test set
        y_pred = best_model.predict(X_test_scaled)
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        # Perform cross-validation on the training set
        cv_score = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='accuracy').mean()
        print(f"Cross-Validation Score: {cv_score:.4f}")

        # Save the best model and scaler
        joblib.dump(best_model, model_path)
        joblib.dump(scaler, scaler_path)
        print(f"\nModel and scaler saved for {ticker} at:\nModel: {model_path}\nScaler: {scaler_path}")

        return best_model, scaler

    except Exception as e:
        raise ValueError(f"Error training and saving model for {ticker}: {e}")




def load_model_and_scaler(model_path, scaler_path):
    """
    Load a trained model and its corresponding scaler from disk.

    Args:
        model_path (str): Path to the saved model file.
        scaler_path (str): Path to the saved scaler file.

    Returns:
        tuple: Loaded model and scaler.
    """
    try:
        model = joblib.load(model_path)
        scaler = joblib.load(scaler_path)
        return model, scaler
    except Exception as e:
        raise ValueError(f"Failed to load model or scaler: {e}")


def analyze_stock(ticker):
    try:
        model_path = f'/content/models/{ticker}_risk_model.pkl'
        scaler_path = f'/content/models/{ticker}_scaler.pkl'

        # Train and save model for the ticker if it doesn't exist
        if not os.path.exists(model_path) or not os.path.exists(scaler_path):
            os.makedirs(os.path.dirname(model_path), exist_ok=True)
            model, scaler = train_and_save_model(ticker, model_path, scaler_path)
        else:
            model, scaler = load_model_and_scaler(model_path, scaler_path)

        # Fetch and process the stock data
        data = get_stock_data(ticker)
        if len(data) < 10:
            raise ValueError(f"Not enough data available for meaningful analysis of {ticker}.")

        data = preprocess_data(data)
        data = add_features(data)
        data = label_risk(data)

        # Get features for prediction
        features = ['Daily Return', 'Volatility', 'MA50', 'MA200']
        latest_data = data[features].iloc[-1]

        # Scale the latest data
        latest_data_scaled = scaler.transform(latest_data.values.reshape(1, -1))

        # Make prediction
        risk_level = model.predict(latest_data_scaled)[0]

        # Format dates and prices for the chart (last 30 days)
        if not isinstance(data.index, pd.DatetimeIndex):
            data.index = pd.to_datetime(data.index)
        dates = [d.strftime('%Y-%m-%d') for d in data.index[-30:]]
        prices = data['Close'].tail(30).tolist()

        # Return the analysis results
        return {
            'risk_level': risk_level,
            'current_price': f"{data['Close'].iloc[-1]:.2f}",
            'volatility': f"{data['Volatility'].iloc[-1]*100:.2f}",
            'daily_return': f"{data['Daily Return'].iloc[-1]*100:.2f}",
            'dates': dates,
            'prices': prices
        }
    except Exception as e:
        logging.error(traceback.format_exc())
        return {'error': str(e)}


# Example usage
if __name__ == "__main__":
    ticker = "AAPL"  # Example ticker
    result = analyze_stock(ticker)
    print(result)


                                 Open        High         Low       Close  \
Date                                                                        
2023-12-14 00:00:00-05:00  197.054623  198.646813  195.203690  197.144180   
2023-12-15 00:00:00-05:00  196.567013  197.432767  196.039598  196.606827   
2023-12-18 00:00:00-05:00  195.134019  195.671395  193.442310  194.934998   
2023-12-19 00:00:00-05:00  195.203693  195.989835  194.935005  195.979889   
2023-12-20 00:00:00-05:00  195.940089  196.716285  193.880188  193.880188   

                              Volume  Dividends  Stock Splits  
Date                                                           
2023-12-14 00:00:00-05:00   66831600        0.0           0.0  
2023-12-15 00:00:00-05:00  128256700        0.0           0.0  
2023-12-18 00:00:00-05:00   55751900        0.0           0.0  
2023-12-19 00:00:00-05:00   40714100        0.0           0.0  
2023-12-20 00:00:00-05:00   52242800        0.0           0.0  

PREPROCESSE

In [3]:
def comprehensive_stock_analysis(tickers):
    """
    Perform comprehensive analysis for multiple stock tickers.

    Args:
        tickers (list): List of stock ticker symbols

    Returns:
        dict: Comprehensive analysis results for each ticker
    """
    results = {}

    for ticker in tickers:
        try:
            data = get_stock_data(ticker)
            if len(data) < 10:
                logging.warning(f"Skipping {ticker} due to insufficient data.")
                results[ticker] = {'error': "Insufficient data for analysis."}
                continue

            # Proceed with analysis
            data = preprocess_data(data)
            data = add_features(data)
            data = label_risk(data)

            # Paths for the model and scaler
            model_path = f'/content/models/{ticker}_risk_model.pkl'
            scaler_path = f'/content/models/{ticker}_scaler.pkl'

            # Load or train the model
            if not os.path.exists(model_path) or not os.path.exists(scaler_path):
                model, scaler = train_and_save_model(ticker, model_path, scaler_path)
            else:
                model, scaler = load_model_and_scaler(model_path, scaler_path)

            # Get features for prediction
            features = ['Daily Return', 'Volatility', 'MA50', 'MA200']
            latest_data = data[features].iloc[-1]

            # Scale the latest data
            latest_data_scaled = scaler.transform(latest_data.values.reshape(1, -1))

            # Make prediction
            risk_level = model.predict(latest_data_scaled)[0]

            # Prepare results
            results[ticker] = {
                'risk_level': risk_level,
                'current_price': f"{data['Close'].iloc[-1]:.2f}",
                'volatility': f"{data['Volatility'].iloc[-1] * 100:.2f}%",
                'daily_return': f"{data['Daily Return'].iloc[-1] * 100:.2f}%",
                'latest_close': data['Close'].iloc[-1],
                'trend': "Bullish" if data['MA50'].iloc[-1] > data['MA200'].iloc[-1] else "Bearish"
            }

        except Exception as e:
            results[ticker] = {
                'error': f"Analysis failed: {str(e)}"
            }

    return results


# **RISK ASSESEMENT OF NSE BSE TOP STOCKS**

In [4]:
# Example usage
if __name__ == "__main__":
    # NSE stocks
    nse_tickers = ['TCS.NS', 'ZOMATO.NS', 'TATASTEEL.NS', 'INFY.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS', 'WAAREEENER.NS', 'KRN.NS']

    # BSE stocks
    bse_tickers = [ ]

    # Combine all tickers
    all_tickers = nse_tickers + bse_tickers

    # Perform comprehensive analysis
    analysis_results = comprehensive_stock_analysis(all_tickers)

    # Print results
    for ticker, result in analysis_results.items():
        print(f"\n{ticker} Analysis:")
        for key, value in result.items():
            print(f"{key.replace('_', ' ').title()}: {value}")


                                  Open         High          Low        Close  \
Date                                                                            
2023-12-13 00:00:00+05:30  3591.791927  3598.150254  3512.386124  3542.502197   
2023-12-14 00:00:00+05:30  3578.434666  3625.752810  3576.068855  3615.155518   
2023-12-15 00:00:00+05:30  3614.613131  3840.606550  3614.613131  3806.153076   
2023-12-18 00:00:00+05:30  3803.294432  3873.187177  3775.741279  3804.378662   
2023-12-19 00:00:00+05:30  3790.380436  3800.337058  3736.161730  3761.989502   

                            Volume  Dividends  Stock Splits  
Date                                                         
2023-12-13 00:00:00+05:30  3850968        0.0           0.0  
2023-12-14 00:00:00+05:30  3290396        0.0           0.0  
2023-12-15 00:00:00+05:30  8531230        0.0           0.0  
2023-12-18 00:00:00+05:30  2521612        0.0           0.0  
2023-12-19 00:00:00+05:30  2004435        0.0           0.0 

ERROR:yfinance:WAAREEENER.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:WAAREEENER.NS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Data for WAAREEENER.NS not available for period '1y'. Trying shorter periods.
Data found for WAAREEENER.NS with period '3mo'.
                                  Open         High          Low        Close  \
Date                                                                            
2024-10-28 00:00:00+05:30  2500.000000  2624.399902  2300.000000  2338.899902   
2024-10-29 00:00:00+05:30  2375.100098  2497.000000  2352.050049  2380.199951   
2024-10-30 00:00:00+05:30  2391.100098  2450.000000  2370.149902  2441.649902   
2024-10-31 00:00:00+05:30  2453.000000  2758.699951  2441.699951  2722.100098   
2024-11-01 00:00:00+05:30  2780.000000  2910.000000  2776.550049  2890.100098   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-10-28 00:00:00+05:30  21535158        0.0           0.0  
2024-10-29 00:00:00+05:30   7937427        0.0           0.0  
2024-10-30 00:00:00+05:30   2849157        0.0        

ERROR:yfinance:KRN.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:KRN.NS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Data for KRN.NS not available for period '1y'. Trying shorter periods.


ERROR:yfinance:KRN.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:KRN.NS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Data found for KRN.NS with period '3mo'.
                                 Open        High         Low       Close  \
Date                                                                        
2024-10-03 00:00:00+05:30  480.000000  513.500000  450.000000  478.790009   
2024-10-04 00:00:00+05:30  474.000000  483.000000  448.549988  452.209991   
2024-10-07 00:00:00+05:30  445.350006  451.399994  402.100006  422.429993   
2024-10-08 00:00:00+05:30  415.000000  438.970001  413.010010  423.179993   
2024-10-09 00:00:00+05:30  423.000000  483.519989  423.000000  459.130005   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-10-03 00:00:00+05:30  33592188        0.0           0.0  
2024-10-04 00:00:00+05:30   5747591        0.0           0.0  
2024-10-07 00:00:00+05:30   4813962        0.0           0.0  
2024-10-08 00:00:00+05:30   1924427        0.0           0.0  
2024-10-09 00:00:00+05:30   4803522      

In [5]:
def risk_analysis_model(new_stock_ticker):
    data = get_stock_data(new_stock_ticker)

    # Check if the data is too short for analysis
    if len(data) < 10:
        logging.warning(f"Skipping {new_stock_ticker} due to insufficient data.")
        results = {'error': "Insufficient data for analysis."}
        return results  # Early return if insufficient data

    # Proceed with analysis if enough data
    data = preprocess_data(data)
    data = add_features(data)
    data = label_risk(data)

    # Paths for the model and scaler
    model_path = f'/content/models/{new_stock_ticker}_risk_model.pkl'
    scaler_path = f'/content/models/{new_stock_ticker}_scaler.pkl'

    # Load or train the model
    if not os.path.exists(model_path) or not os.path.exists(scaler_path):
        model, scaler = train_and_save_model(new_stock_ticker, model_path, scaler_path)
    else:
        model, scaler = load_model_and_scaler(model_path, scaler_path)

    # Get features for prediction
    features = ['Daily Return', 'Volatility', 'MA50', 'MA200']
    latest_data = data[features].iloc[-1]

    # Scale the latest data
    latest_data_scaled = scaler.transform(latest_data.values.reshape(1, -1))

    # Make prediction
    risk_level = model.predict(latest_data_scaled)[0]

    # Prepare results
    results = {
        'risk_level': risk_level,
        'current_price': f"{data['Close'].iloc[-1]:.2f}",
        'volatility': f"{data['Volatility'].iloc[-1] * 100:.2f}%",
        'daily_return': f"{data['Daily Return'].iloc[-1] * 100:.2f}%",
        'latest_close': data['Close'].iloc[-1],
        'trend': "Bullish" if data['MA50'].iloc[-1] > data['MA200'].iloc[-1] else "Bearish"
    }

    return results


In [6]:

def fetch_risk_results(new_stock_ticker, portfolio):
    print(new_stock_ticker)

    if new_stock_ticker not in portfolio:
        portfolio.append(new_stock_ticker)
        model_path = f'/content/models/{new_stock_ticker}_risk_model.pkl'
        scaler_path = f'/content/models/{new_stock_ticker}_scaler.pkl'

        model, scaler = train_and_save_model(new_stock_ticker, model_path, scaler_path)
        print(f"Model trained for {new_stock_ticker}...")

        load_model_and_scaler(model_path, scaler_path)
        print(f"Loaded trained model for {new_stock_ticker}...")

        results = risk_analysis_model(new_stock_ticker)
        return results
    else:
        model_path = f'/content/models/{new_stock_ticker}_risk_model.pkl'
        scaler_path = f'/content/models/{new_stock_ticker}_scaler.pkl'
        print(f"{new_stock_ticker} is already in the portfolio.")

        model, scaler = load_model_and_scaler(model_path, scaler_path)

        results = risk_analysis_model(new_stock_ticker)
        return results

ITC STOCK RISK ASSESSMENT

In [7]:
portfolio = ['TCS.NS',  'TCS.NS', 'ZOMATO.NS', 'TATASTEEL.NS', 'INFY.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS']

new_stock_tickerr = 'ITC.NS'

results = fetch_risk_results(new_stock_tickerr, portfolio)

print(results)

ITC.NS
                                 Open        High         Low       Close  \
Date                                                                        
2023-12-13 00:00:00+05:30  441.858212  444.327523  438.759509  441.228790   
2023-12-14 00:00:00+05:30  445.441153  446.409504  435.225069  445.537994   
2023-12-15 00:00:00+05:30  447.280999  448.297784  440.599387  443.698120   
2023-12-18 00:00:00+05:30  440.599387  441.567737  436.096564  437.355408   
2023-12-19 00:00:00+05:30  439.001618  443.504441  437.742745  441.664581   

                             Volume  Dividends  Stock Splits  
Date                                                          
2023-12-13 00:00:00+05:30  11283868        0.0           0.0  
2023-12-14 00:00:00+05:30  14370803        0.0           0.0  
2023-12-15 00:00:00+05:30  15789845        0.0           0.0  
2023-12-18 00:00:00+05:30  12027620        0.0           0.0  
2023-12-19 00:00:00+05:30  10268755        0.0           0.0  

PREPROCESSE

CIPLA STOCK RISK ASSESSMENT

In [8]:
portfolio = ['TCS.NS', 'ITC.NS', 'TCS.NS', 'ZOMATO.NS', 'TATASTEEL.NS', 'INFY.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS']

new_stock_tickerr = 'CIPLA.NS'

results = fetch_risk_results(new_stock_tickerr, portfolio)

print(results)

CIPLA.NS
                                  Open         High          Low        Close  \
Date                                                                            
2023-12-13 00:00:00+05:30  1188.572075  1207.709733  1185.151009  1206.271973   
2023-12-14 00:00:00+05:30  1208.800682  1212.271255  1191.001527  1194.521729   
2023-12-15 00:00:00+05:30  1204.784653  1207.362769  1192.736782  1196.951050   
2023-12-18 00:00:00+05:30  1195.463673  1208.701407  1182.077177  1206.569580   
2023-12-19 00:00:00+05:30  1209.197224  1229.723258  1202.008179  1226.698853   

                            Volume  Dividends  Stock Splits  
Date                                                         
2023-12-13 00:00:00+05:30  1809563        0.0           0.0  
2023-12-14 00:00:00+05:30  1809248        0.0           0.0  
2023-12-15 00:00:00+05:30  1976112        0.0           0.0  
2023-12-18 00:00:00+05:30  1995091        0.0           0.0  
2023-12-19 00:00:00+05:30  1738507        0.0      

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
!pip install huggingface

In [ ]:
from huggingface_hub import InferenceClient


from google.colab import userdata


client = InferenceClient(api_key=userdata.get('HF_TOKEN'))


In [ ]:

from huggingface_hub import InferenceClient
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Authenticate Hugging Face API Key
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
client = InferenceClient(api_key=hf_token)

# Load a smaller, more manageable model for local inference
# Consider using a smaller model like 'distilgpt2' for faster performance
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # You can change this to your desired model if it's available on Hugging Face

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else use CPU
)

# Define the system and user messages
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Prepare the prompt and generate the response
prompt = messages[-1]['content']
outputs = pipe(prompt, max_length=100, num_return_sequences=1)

# Display the generated text
print(outputs[0]["generated_text"])



OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct.
403 Client Error. (Request ID: Root=1-675c5c48-55c1a0df11ecc30b7310d58f;90247053-fb66-44e8-a7f4-915674e4a77e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-3B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct to ask for access.